In [ ]:
import pandas as pd
from datetime import datetime, timedelta

from src import *
pd.set_option('display.max_rows', None)

# Parameters

These parameters configure various things such as the relative location of data files, which date is being analysed
and the assumed generation period for an infection.

In [ ]:
ALL_VIC_CASES='archive/2021-07-12/all-vic-cases.csv'
QUARANTINE='archive/2021-07-16/quarantine.csv'
GENERATION_DAYS=5
ENABLE_MELBOURNE_ANIMATION=False

TODAY=datetime.today().strftime("%Y-%m-%d")
LAST_WEEK=add_days(TODAY, -7)

RUN_DATE=TODAY
#RUN_DATE='2021-07-27'
PREV_DAY=add_days(RUN_DATE, -1)
NEXT_WEEK=add_days(RUN_DATE, 7)
PREV_WEEK=add_days(RUN_DATE, -7)
PREV_FORTNIGHT=add_days(RUN_DATE, -14)

NSW_6_MONTHS=f"archive/{RUN_DATE}/last-6-months-nsw.csv"
NSW_14_DAYS=f"archive/{RUN_DATE}/last-14-days-nsw.csv"

# Data Preparation

**sweep_downloads()** moves files from the $HOME/Downloads directory into today's archive directory. Thses should first be
downloaded from the "Last 14 days (new)" and "Last 6 months (true)" panels of the NSW Transmission Sources section of [covid19data.com.au](https://www.covid19data.com.au/nsw)

In [ ]:
sweep_downloads(TODAY)

Next, we load the data from the files and merge the 14-day data with the 6 month data.

In [ ]:
nsw_df = update_df(load_data(NSW_6_MONTHS),load_data(NSW_14_DAYS))
vic_df = load_vic_data(ALL_VIC_CASES)
quarantine_df = load_quarantine(QUARANTINE)

Next, we truncate and refindex the data frames to each outbreak.

In [ ]:
avalon = select_outbreak(nsw_df[(nsw_df['date'] >= '2020-12-17') & (nsw_df['date'] <= '2021-01-16')], generation_days=GENERATION_DAYS)
bondi = select_outbreak(nsw_df[(nsw_df['date'] >= '2021-06-17')], generation_days=GENERATION_DAYS)
vic_outbreak = select_outbreak(vic_df[(vic_df['date'] >= '2020-05-27') & (vic_df['date'] <= '2020-10-29')], generation_days=GENERATION_DAYS)

# Development

In [ ]:
df=vic_outbreak
offset=30
data=[]
for x in range(offset+5, len(df)):
    slice=df[df.index < x]
    gp=derive_growth_params(slice[slice.index>=offset])
    prj=select_outbreak(project_ols_growth_rate_min(slice, 365-x, gp[1]))
    max_total = prj['total'].max()
    max_cumulative= prj['cumulative'].max()
    peak=prj[np.abs(prj.total-max_total) < 1e-3].index
    if len(peak) > 0:
        peak_day = peak.values[0]
    else:
        peak_day = len(prj)
    last=prj[(prj.index > peak_day) & (prj.total < 2)].index
    if len(last) > 0:
        last_day = last.values[0]
    else:
        last_day = len(prj)
        
    decay_rate = (gp[1]-1)*100
    
    data.append([x-1, max_total, max_cumulative, peak_day, last_day, decay_rate])
data=np.array(data)
out=pd.DataFrame(
    index=np.int32(data[:,0]), 
    columns=[
        'max_total', 
        'max_cumulative', 
        'peak_day', 
        'last_day', 
        'decay_rate'
    ], 
    data=data[:,1:])


In [ ]:
_=out[(out.max_total < 10000) & (out.max_cumulative < 150000)].max_total.plot()

In [ ]:
_=out[(out.max_total < 10000) & (out.max_cumulative < 150000)].max_cumulative.plot()

In [ ]:
_=out[(out.max_total < 10000) & (out.max_cumulative < 150000)].decay_rate.plot()

In [ ]:
out[(out.max_total < 10000) & (out.max_cumulative < 150000)]

In [ ]:
out['peak_day'].plot()

In [ ]:
out['last_day'].plot()

In [ ]:
add_days('2021-06-17', 85)

In [ ]:
add_days('2021-06-17', 344)